In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score, roc_curve

from imblearn.under_sampling import RandomUnderSampler

from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

pd.set_option('precision', 3) # set precision to 2 decimal points
%matplotlib inline

In [2]:
file_path = "./sao-paulo-properties-april-2019.csv"
df = pd.read_csv(file_path)

In [3]:
sp_lat = -23.533773
sp_lon = -46.625290

In [4]:
df['Distance'] = (df['Longitude'] - sp_lon)**2 + (df['Latitude'] - sp_lat)**2

In [6]:
df_norm = df.copy()

std_scaler = StandardScaler()
feature_list = ['Price', 'Condo','Size','Distance']

for i in feature_list:
    df_norm[i] = std_scaler.fit_transform(df_norm[i].values.reshape(-1, 1))





In [7]:
df_norm.head()

,Price,Condo,Size,Rooms,Toilets,Suites,Parking,Elevator,Furnished,Swimming Pool,New,District,Negotiation Type,Property Type,Latitude,Longitude,Distance
0,-0.485,-0.620,-0.640,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.543,-46.479,-0.265
1,-0.485,-0.715,-0.674,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.550,-46.481,-0.265
2,-0.485,-0.779,-0.622,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.543,-46.486,-0.265
3,-0.485,-0.647,-0.622,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.547,-46.483,-0.265
4,-0.485,-0.369,-0.503,2,2,1,1,1,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.525,-46.482,-0.265
